In [1]:
# imports
from IPython.display import clear_output
from PIL import Image
import os
import json
# clear
clear_output(wait=True)
# installing kaggle
!pip install -q kaggle
# user and key info
info = {"username":"jackbellamy","key":"3442898ba5e904ac3a48eb5b59398805"}
# opening file
json_object = json.dumps(info, indent=4)
with open("kaggle.json", "w") as outfile:
    outfile.write(json_object)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# copying data to content file of kaggle which can be found on the left hand side of the screen
if not (os.path.exists("/content/nfl-big-data-bowl-2023")):
  !kaggle competitions download -c nfl-big-data-bowl-2023
  !mkdir nfl-big-data-bowl-2023
  !unzip nfl-big-data-bowl-2023.zip -d nfl-big-data-bowl-2023

 95% 163M/172M [00:02<00:00, 79.0MB/s]
100% 172M/172M [00:02<00:00, 68.6MB/s]
Archive:  nfl-big-data-bowl-2023.zip
  inflating: nfl-big-data-bowl-2023/games.csv  
  inflating: nfl-big-data-bowl-2023/pffScoutingData.csv  
  inflating: nfl-big-data-bowl-2023/players.csv  
  inflating: nfl-big-data-bowl-2023/plays.csv  
  inflating: nfl-big-data-bowl-2023/week1.csv  
  inflating: nfl-big-data-bowl-2023/week2.csv  
  inflating: nfl-big-data-bowl-2023/week3.csv  
  inflating: nfl-big-data-bowl-2023/week4.csv  
  inflating: nfl-big-data-bowl-2023/week5.csv  
  inflating: nfl-big-data-bowl-2023/week6.csv  
  inflating: nfl-big-data-bowl-2023/week7.csv  
  inflating: nfl-big-data-bowl-2023/week8.csv  


In [2]:
#import the libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.impute import SimpleImputer

In [3]:
#read csv files
dfPlays = pd.read_csv('/content//nfl-big-data-bowl-2023/plays.csv')
dfPlayer = pd.read_csv('/content/nfl-big-data-bowl-2023/players.csv')
dfWeek = pd.read_csv('/content/nfl-big-data-bowl-2023/week1.csv')

In [4]:
dfPlays = pd.merge(dfWeek, dfPlays, on=['playId'])

def convertTimeToNumerical2(t):
    t = t.split(":")
    if len(t) == 2:
        return int(t[0])
    return int(t[0])

dfPlays['gameClock_new'] = dfPlays['gameClock'].apply(convertTimeToNumerical2)

In [5]:
# Selecting train and test data
#xInput = dfPlays[['defendersInBox','quarter','yardsToGo','gameClock_new']]#,'gameClock']]#,'height','s']]

plays = dfPlays

I_data = plays.loc[plays['passResult']=="I"]
C_data = plays.loc[plays['passResult']=="C"]
IN_data = plays.loc[plays['passResult']=="IN"]
S_data = plays.loc[plays['passResult']=="S"]

all_data = pd.concat([C_data, I_data, IN_data, S_data], axis=0)

labels = np.array([0]*len(C_data)+[1]*len(I_data)+[2]*len(IN_data)+[3]*len(S_data))

xInput = all_data[['quarter','down','yardsToGo','absoluteYardlineNumber','gameClock_new']]

yInput = labels
#yInput.dropna()
xInput.head()

,quarter,down,yardsToGo,absoluteYardlineNumber,gameClock_new
2,1,1,10,72.0,13
4,1,3,4,41.0,13
5,1,3,2,77.0,13
8,1,1,10,72.0,13
10,1,3,4,41.0,13


In [17]:
#split data into train and test
#week1
x_train,x_test,y_train,y_test = train_test_split(xInput,yInput,test_size=0.25) #random_state=0
#scaling dataset
scaler = MinMaxScaler()
xTrainScaled = scaler.fit_transform(x_train)
xTestScaled = scaler.fit_transform(x_test)

print(xTestScaled)

[[0.5        0.75       0.13333333 0.94897959 0.66666667]
 [0.25       0.5        0.33333333 0.71428571 0.06666667]
 [0.25       0.75       0.2        0.13265306 0.93333333]
 ...
 [0.75       0.5        0.06666667 0.65306122 0.66666667]
 [0.5        0.25       0.33333333 0.43877551 0.13333333]
 [0.25       0.5        0.26666667 0.32653061 0.53333333]]


In [18]:
#select model
model = DecisionTreeClassifier(max_depth=15)
#train model
model.fit(xTrainScaled,y_train)

predictions = model.predict(xTrainScaled)
#observing performance metrics
accuracy = accuracy_score(y_train,predictions)*100
print("Accuracy of the model is {:.2f}%".format(accuracy))

predictions = model.predict(xTestScaled)
#observing performance metrics
accuracy = accuracy_score(y_test,predictions)*100
print("Accuracy of the model is {:.2f}%".format(accuracy))

print("-----------------")
print(y_test)
print(predictions)
print("-----------------")
#y_pred = np.argmax(y_test, axis=1)
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_test, predictions)

print(confusion_matrix)

Accuracy of the model is 82.28%
Accuracy of the model is 82.36%
-----------------
[1 0 3 ... 0 0 1]
[1 0 3 ... 0 0 1]
-----------------
[[437179  16985    826   1769]
 [ 91132 173066    410   1723]
 [  6196   1469  10468    500]
 [ 17824   2674    169  40673]]


In [26]:
data = [[4,4,13,44,7]]

scaler = MinMaxScaler()

xdata = scaler.fit_transform(data)

predictions = model.predict(data)

print(predictions)

[2]


In [ ]:
from sklearn import tree
text_representation = tree.export_text(model)
print(text_representation)

In [19]:
print(model.tree_.max_depth)
print(model.tree_.node_count)

15
1393
